## **구글 드라이브 마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **외부 라이브러리 설치**

In [ ]:
# transformer: NLP model
# evaluate: metric for F1 score
!pip install transformers evaluate

## **Import Libraries**

In [ ]:
import os
import json
import torch
import random
import shutil
import evaluate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.auto import tqdm
from torch.optim import AdamW
from pytorch_optimizer import Ranger21
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, get_scheduler

## **GPU Setting**

In [ ]:
# GPU 사용을 위해 cuda 사용이 가능한지 확인합니다.
# 'CPU'라고 나타나는 경우, colab 런타임 유형 변경을 통해 바꾸셔야 합니다. 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'device:{device}')

device:cuda


## **Hyperparameter Setting**

In [ ]:
# model
checkpoint = 'bert-base-uncased'
num_labels = 5
model_path = f'model_with_{checkpoint}'

# training
use_amp = True
num_epochs = 100
lr = 1e-5 # 학습 양상이 처음부터 계속 valid loss 가 증가하는 방향이라 2e-5 에서 낮추게 되었습니다.
batch_size = 32

# scheduler
scheduler_name = 'linear'
num_warmup_steps = 0

# early_stop
early_stop = 3 # 좋아지는 경우가 거의 없어서 GPU 사용량을 효율적으로 운영하고자 값을 줄였습니다.

# seed
seed = 2022

## **Seed Setting**

- 최대한 REPRODUCIBILITY 를 보장하기 위해 제어합니다.
- 단, pytorch 역시 공식적으로 완전히 보장할 수 없다고 말합니다.

- pytorch link: https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
def seed_everything(seed: int = 42, contain_cuda: bool = False):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print(f"Seed set as {seed}")

In [ ]:
seed_everything(seed=2022)

Seed set as 2022


## **Load Original CSV File**

In [ ]:
def dataframe_from_csv(target):
      return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

In [ ]:
data_path = '/content/drive/MyDrive/dataset'

train_files = sorted([x for x in Path(f'{data_path}/train/').glob('*.csv')])
val_files = sorted([x for x in Path(f'{data_path}/val/').glob('*.csv')])

train = dataframe_from_csvs(train_files)
val = dataframe_from_csvs(val_files)
test = pd.read_csv(f'{data_path}/test.csv')
print(f'train: {len(train)}')
print(f'validation: {len(val)}')
print(f'test: {len(test)}')

In [ ]:
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

## **Normalize Dataset**

In [ ]:
def normalize(df):
    """
    column 별로 정규화를 시키는 함수입니다.
    이때, 정규화 방식은 표준편차가 아닌 최대 - 최소로 구하였습니다.
    진행한 이유는 특정 column 값이 큰 영향을 끼칠 수 없도록 제한하기 위함입니다.

    Args:
        df (DataFrame): csv 파일을 pandas 로 읽은 데이터

    Returns:
        result (DataFrame) : df 를 column 별로 정규화한 데이터
    """
    result = df.copy()
    
    for feature_name in df.columns:
        # site, sid, leaktype 은 정규화 대상이 아니므로, 값을 그대로 저장하고 넘어갑니다.
        if feature_name in ['site', 'sid', 'leaktype']:
            result[feature_name] = df[feature_name]
            continue
    
        # C01 ~ C26
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value) * 100
        
    return result

In [ ]:
def num2alpha(char_dict):
    """
    숫자를 자연어(영어 형태)로 변환하는 함수입니다.
    
    영어를 택한 이유는 크게 2가지입니다.
    - C01 ~ C26 이 26개로 알파벳 개수와 동일하다.
    - 현재 NLP 에서 가장 뛰어난 모델은 대부분 영어 데이터에 최적화되어 있다.
    
    결론부터 말씀드리면 결과는 다음 예시처럼 나옵니다.
    
    예시: 'site: S-4784025026. sid: S-0359369085186035. aa.... zz.'
    
    변환하는 방식은 다음과 같습니다.
    
    1. 각 row 별로 백분율을 구합니다.
    2. 그리고 백분율에 400 을 곱합니다.
    여기서 400 을 곱하는 이유는 'BERT'의 최대 입력 길이를 고려했기 때문입니다.
    최대 길이가 512 인데, site 와 sid 를 제일 앞에 표기한 길이를 대강 112 로 정했습니다.
    그리고 그 후, 나머지 길이 400 을 나머지 row 끼리 나눠서 알파벳을 나열하는 구조입니다.
    3. 이때, 알파벳별로 간격을 두었습니다.

    Args:
        char_dict (Dict): C01 ~ C26 값을 모두 저장한 딕셔너리 자료형

    Returns:
        str : 자연어로 변환된 문자열
    """
    sentence = list()
    
    s = sum(char_dict.values())
    for k, v in char_dict.items():
        a = round(v / s * 400) * chr(96 + int(k[1:]))
        sentence.append(a)

    return ' '.join(sentence)
    

In [ ]:
def num2alpha(char_dict):
    """
    숫자를 자연어(영어 형태)로 변환하는 함수입니다.
    
    영어를 택한 이유는 크게 2가지입니다.
    - C01 ~ C26 이 26개로 알파벳 개수와 동일하다.
    - 현재 NLP 에서 가장 뛰어난 모델은 대부분 영어 데이터에 최적화되어 있다.
    
    결론부터 말씀드리면 결과는 다음 예시처럼 나옵니다.
    
    예시: 'site: S-4784025026. sid: S-0359369085186035. aa.... zz.'
    
    변환하는 방식은 다음과 같습니다.
    
    1. 각 row 별로 백분율을 구합니다.
    2. 그리고 백분율에 400 을 곱합니다.
    여기서 400 을 곱하는 이유는 'BERT'의 최대 입력 길이를 고려했기 때문입니다.
    최대 길이가 512 인데, site 와 sid 를 제일 앞에 표기한 길이를 대강 112 로 정했습니다.
    그리고 그 후, 나머지 길이 400 을 나머지 row 끼리 나눠서 알파벳을 나열하는 구조입니다.
    3. 이때, 알파벳별로 간격을 두었습니다.

    Args:
        char_dict (Dict): C01 ~ C26 값을 모두 저장한 딕셔너리 자료형

    Returns:
        str : 자연어로 변환된 문자열
    """
    sentence = list()
    
    s = sum(char_dict.values())
    for k, v in char_dict.items():
        a = round(v / s * 400) * chr(96 + int(k[1:]))
        sentence.append(a)

    return ' '.join(sentence)
    

## **Tabular Dataset to Natural Language Dataset**

In [ ]:
def data2nlp(df):
    """
    실질적으로 부르는 함수.
    1. 정규화하고,
    2. 자연어로 변환한다.
    
    간단히 표현하고자 있는 함수입니다.
    """
    df = normalize(df)
    return tablet2nlp(df)

In [ ]:
# 데이터를 모두 자연어 형태로 변환한다.
nlp_train = data2nlp(train)
nlp_val = data2nlp(val)
nlp_test = data2nlp(test)

In [ ]:
# 데이터를 모두 저정해둡니다. 이때, index=False
nlp_train.to_csv('dataset/nlp/train.csv', index=False)
nlp_val.to_csv('dataset/nlp/val.csv', index=False)
nlp_test.to_csv('dataset/nlp/test.csv', index=False)

## **Load CSV Natural Language file**

In [ ]:
data_path = 'dataset/nlp'

train_csv = pd.read_csv(os.path.join(data_path, 'train.csv'))
valid_csv = pd.read_csv(os.path.join(data_path, 'val.csv'))
test_csv = pd.read_csv(os.path.join(data_path, 'test.csv'))

## **Customize Dataset**

In [ ]:
class UnidthonDataset(Dataset):
  def __init__(self, csv, tokenizer):
    self.dataset = tokenizer(list(csv['nlp']), truncation=True, padding=True, return_tensors='pt')
    self.labels = list(csv['leaktype'])
  
  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, idx):
    item = {k: torch.tensor(v[idx]) for k, v in self.dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])

    item = {k: torch.tensor(v[idx]) for k, v in self.dataset.items()}
    if np.isnan(self.labels[idx]): # test dataset 은 값이 비어 있어, NAN 을 방지하기 위해 '0'을 기입합니다.
        item['labels'] = 0
        
    else: 
        item['labels'] = torch.tensor(self.labels[idx])

    return item

## **Load Tokenizer**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## **CSV to DataLoader**

In [ ]:
def csv_to_dataloader(csv, tokenizer, mode, batch_size):
  shuffle = True if mode in ['train', 'valid'] else False
  dataset = UnidthonDataset(csv, tokenizer)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

  return dataloader

In [ ]:
# dataset
train_dataset = UnidthonDataset(train_csv, tokenizer)
valid_dataset = UnidthonDataset(valid_csv, tokenizer)
test_dataset = UnidthonDataset(test_csv, tokenizer)

In [ ]:
# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # test should not be shuffled !

## **Load Model**

In [ ]:
# model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                            num_labels=num_labels)
model.to(device)

## **Load Optimizer & Scheduler**

In [ ]:
# optimizer & scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name=scheduler_name, optimizer=optimizer,
                              num_warmup_steps=num_warmup_steps,
                              num_training_steps=num_training_steps)

## **Make Method to infer per epoch**

- 코드의 간결화를 위해 train, eval 모두 하나의 함수에서 가능하도록 만들었습니다.

In [ ]:
def inference_per_epoch(tokenizer, model, optimizer, scheduler, use_amp, scaler, dataloader, mode):
  if mode == 'train':
    model.train()
  
  else: # 'valid' or 'test'
    model.eval()

  total_loss = 0
  metric = evaluate.load('f1')

  for batch in tqdm(dataloader, desc=f'{mode} per epoch'):
    batch = { k: v.to(device) for k, v in batch.items() }

    if mode == 'train':
      # colab 인 점을 고려하여, GPU 사용량 및 시간을 절약하고자 사용합니다.
      with torch.cuda.amp.autocast(enabled=use_amp):
        outputs = model(**batch)
        loss = outputs.loss
      
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()

      scheduler.step()
      optimizer.zero_grad()

    else:
      with torch.no_grad():
        outputs = model(**batch)

      loss = outputs.loss

    total_loss += loss.item()
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

  total_loss /= len(dataloader)
  f1 = metric.compute(average='micro')['f1']

  return total_loss, f1

## **Training Loops**

In [ ]:
best_model_path = 'best_model'
best_performance = 0
best_epoch = 0
patience = 0 # for Early Stop

scaler = GradScaler(enabled=use_amp)

for epoch in tqdm(range(num_epochs), desc='Training loops'):
  # train per epoch
  train_loss, train_f1 = inference_per_epoch(tokenizer=tokenizer, model=model,
                                              optimizer=optimizer, scheduler=lr_scheduler,
                                              use_amp=use_amp, scaler=scaler,
                                              dataloader=train_dataloader, mode='train')

  # evaluate per epoch
  valid_loss, valid_f1 = inference_per_epoch(tokenizer=tokenizer, model=model,
                                              optimizer=optimizer, scheduler=lr_scheduler,
                                              use_amp=use_amp, scaler=scaler,
                                              dataloader=valid_dataloader, mode='valid')

  print(f'epoch: {epoch}')
  print(f'train_loss: {train_loss}, train_f1: {train_f1}')
  print(f'valid_loss: {valid_loss}, valid_f1: {valid_f1}\n')

  if best_performance < valid_f1:
    patience = 0

    print(f'best performance: {best_performance} → {valid_f1}\n')

    # save best model
    if os.path.exists(best_model_path):
      shutil.rmtree(best_model_path) # delete everything in the directory

    model.save_pretrained(best_model_path)

    best_performance = valid_f1
    best_epoch = epoch

    print(f'saving best model at epoch: {epoch}')
    print(f'best performance: {best_performance} → {valid_f1}\n')

  else:
    patience += 1

  # early_stop
  if early_stop <= patience:
    print('----- Early Stop -----')
    print(f'→ number of patience: {patience}\n')
    break

## **Test the model**

In [ ]:
# load best model
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path, num_labels=num_labels)
best_model.to(device)
best_model.eval()

# inference
answer_list = list()

for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad():
        outputs = best_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    answer_list += predictions.detach().cpu().tolist()

## **Save the result**

In [ ]:
result_csv = pd.DataFrame()
result_csv['site'] = test['site']
result_csv['sid'] = test['sid']
result_csv['leaktype'] = answer_list

result_csv.to_csv('submission.csv', index=False)

In [ ]:
result_csv.head()